# Coder Hussam Qassim

In [1]:
'''
First, let's make sure this notebook works well in both python 2 and 3, import a few common modules, ensure 
MatplotLib plots figures inline and prepare a function to save the figures
'''
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ensembles"

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id)

def save_fig(fig_id, tight_layout=True):
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(image_path(fig_id) + ".png", format='png', dpi=300)

# Preprocessing the Dataset 

In [2]:
# Import the Dataset 
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Ensemble

In [ ]:
'''
The following code creates and trains a voting classifier in Scikit-Learn, composed of three diverse classifiers
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()
voting_clf = VotingClassifier(
                        estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
                        voting='hard'
                    )
voting_clf.fit(X_train, y_train)

# Let’s look at	each classifier’s accuracy on the test set
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
    
'''
A very simple way to create an even better classifier is to aggregate the predictions of each classifier and
predict the class that gets the most votes. This majority-vote classifier is called a hard voting classifier.
There you have it! The voting classifier slightly outperforms all the individual classifiers. If all 
classifiers are able to estimate class probabilities (i.e., they have a predict_proba() method), then you can
tell Scikit-Learn to predict the class with the highest class probability, averaged over all the individual 
classifiers. This is called soft voting. It often achieves higher performance than hard voting because it gives
more weight to highly confident votes. All you need to do is replace voting="hard" with voting="soft" and 
ensure that all classifiers can estimate class probabilities. This is not the case of the SVC class by default,
so you need to set its probability hyperparameter to True (this will make the SVC class use cross-validation
to estimate class probabilities, slowing down training, and it will add a predict_proba() method). If you 
modify the preceding code to use soft voting, you will find that the voting classifier achieves over 91% 
accuracy!
'''

# Bagging and Pasting

In [3]:
'''
One way to get a diverse set of classifiers is to use very different training algorithms, as just discussed.
Another approach is to use the same training algorithm for every predictor, but to train them on different
random subsets of the training set. When sampling is performed with replacement, this method is called bagging
(short for bootstrap aggregating). When sampling is performed without replacement, it is called pasting. In 
other words, both bagging and pasting allow training instances to be sampled several times across multiple 
predictors, but only bagging allows training instances to be sampled several times for the same predictor. 
Once all predictors are trained, the ensemble can make a prediction for a new instance by simply aggregating
the predictions of all predictors. The aggregation function is typically the statistical mode (i.e., the most
frequent prediction, just like a hard voting classifier) for classification, or the average for regression. 
Each individual predictor has a higher bias than if it were trained on the original training set, but 
aggregation reduces both bias and variance. Generally, the net result is that the ensemble has a similar bias
but a lower variance than a single predictor trained on the original training set. Predictors can all be 
trained in parallel, via different CPU cores or even different servers. Similarly, predictions can be made in
parallel. This is one of the reasons why bagging and pasting are such popular methods: they scale very well.
'''

'''
Scikit-Learn offers a simple API for both bagging and pasting with the BaggingClassifier class (or 
BaggingRegressor for regression). The following code trains an ensemble of 500 Decision Tree classifiers, 5 
each trained on 100 training instances randomly sampled from the training set with replacement (this is an 
example of bagging, but if you want to use pasting instead, just set bootstrap=False). The n_jobs parameter 
tells Scikit-Learn the number of CPU cores to use for training and predictions (–1 tells Scikit-Learn to use
all available cores)
'''
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
bag_clf = BaggingClassifier(
                        DecisionTreeClassifier(), n_estimators=500,
                        max_samples=100, bootstrap=True, n_jobs=-1
                    )
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

'''
NOTE: The BaggingClassifier automatically performs soft voting instead of hard voting if the base classifier
can estimate class probabilities (i.e., if it has a predict_proba() method), which is the case with Decision
Trees classifiers.
'''

'\nNOTE: The BaggingClassifier automatically performs soft voting instead of hard voting if the base classifier\ncan estimate class probabilities (i.e., if it has a predict_proba() method), which is the case with Decision\nTrees classifiers.\n'

# Out-of-Bag Evaluation

In [8]:
'''
With bagging, some instances may be sampled several times for any given predictor, while others may not be 
sampled at all. By default a BaggingClassifier samples m training instances with replacement ( bootstrap=True ),
where m is the size of the training set. This means that only about 63% of the training instances are sampled
on average for each predictor. The remaining 37% of the training instances that are not sampled are called 
out-of-bag (oob) instances. Note that they are not the same 37% for all predictors. Since a predictor never 
sees the oob instances during training, it can be evaluated on these instances, without the need for a separate
validation set or cross-validation. You can evaluate the ensemble itself by averaging out the oob evaluations
of each predictor. In Scikit-Learn, you can set oob_score=True when creating a BaggingClassifier to request an
automatic oob evaluation after training. The following code demonstrates this. The resulting evaluation score
is available through the oob_score_ variable
'''
bag_clf = BaggingClassifier(
                        DecisionTreeClassifier(), n_estimators=500,
                        bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
print('bag_clf.oob_score: ', bag_clf.oob_score_)

'''
According to this oob evaluation, this BaggingClassifier is likely to achieve about 89,86% accuracy on the 
test set,
'''
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
print('Accuracy_score: ',accuracy_score(y_test, y_pred))

'''
The oob decision function for each training instance is also available through the oob_decision_function_ 
variable. In this case (since the base estimator has a predict_proba() method) the decision function returns 
the class probabilities for each training instance. For example, the oob evaluation estimates that the second
training instance has a 60.6% probability of belonging to the positive class (and 39.4% of belonging to the
positive class)
'''
print('bag_clf.oob_decision_function: ', bag_clf.oob_decision_function_)

bag_clf.oob_score:  0.904
Accuracy_score:  0.904
bag_clf.oob_decision_function:  [[ 0.38251366  0.61748634]
 [ 0.32432432  0.67567568]
 [ 1.          0.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 [ 0.0877193   0.9122807 ]
 [ 0.37777778  0.62222222]
 [ 0.03932584  0.96067416]
 [ 0.98734177  0.01265823]
 [ 0.96808511  0.03191489]
 [ 0.73493976  0.26506024]
 [ 0.00591716  0.99408284]
 [ 0.7970297   0.2029703 ]
 [ 0.83333333  0.16666667]
 [ 0.97409326  0.02590674]
 [ 0.085       0.915     ]
 [ 0.          1.        ]
 [ 0.99408284  0.00591716]
 [ 0.94054054  0.05945946]
 [ 0.96666667  0.03333333]
 [ 0.01149425  0.98850575]
 [ 0.3258427   0.6741573 ]
 [ 0.90909091  0.09090909]
 [ 1.          0.        ]
 [ 0.9673913   0.0326087 ]
 [ 0.          1.        ]
 [ 1.          0.        ]
 [ 1.          0.        ]
 [ 0.          1.        ]
 [ 0.59562842  0.40437158]
 [ 0.          1.        ]
 [ 1.          0.        ]
 [ 0.          1.        ]
 [ 0.          1.        ]
 

# Random Patches and Random Subspaces

In [9]:
'''
The BaggingClassifier class supports sampling the features as well. This is controlled by two hyperparameters:
max_features and bootstrap_features . They work the same way as max_samples and bootstrap, but for feature 
sampling instead of instance sampling. Thus, each predictor will be trained on a random subset of the input 
features. This is particularly useful when you are dealing with high-dimensional inputs (such as images). 
Sampling both training instances and features is called the Random Patches method. Keeping all training 
instances (i.e., bootstrap=False and max_samples=1.0 ) but sampling features (i.e., bootstrap_features=True
and/or max_features smaller than 1.0) is called the Random Subspaces method. Sampling features results in even
more predictor diversity, trading a bit more bias for a lower variance.
'''

'\nThe BaggingClassifier class supports sampling the features as well. This is controlled by two hyperparameters:\nmax_features and bootstrap_features . They work the same way as max_samples and bootstrap, but for feature \nsampling instead of instance sampling. Thus, each predictor will be trained on a random subset of the input \nfeatures. This is particularly useful when you are dealing with high-dimensional inputs (such as images). \nSampling both training instances and features is called the Random Patches method. Keeping all training \ninstances (i.e., bootstrap=False and max_samples=1.0 ) but sampling features (i.e., bootstrap_features=True\nand/or max_features smaller than 1.0) is called the Random Subspaces method. Sampling features results in even\nmore predictor diversity, trading a bit more bias for a lower variance.\n'

# Random Forests

In [10]:
'''
As we have discussed, a Random Forest is an ensemble of Decision Trees, generally trained via the bagging 
method (or sometimes pasting), typically with max_samples set to the size of the training set. Instead of 
building a BaggingClassifier and passing it a DecisionTreeClassifier , you can instead use the 
RandomForestClassifier class, which is more convenient and optimized for Decision Trees (similarly, there is 
a RandomForestRegressor class for regression tasks). The following code trains a Random Forest classifier with
500 trees (each limited to maximum 16 nodes), using all available CPU cores
'''
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

'''
With a few exceptions, a RandomForestClassifier has all the hyperparameters of a DecisionTreeClassifier (to 
control how trees are grown), plus all the hyperparameters of a BaggingClassifier to control the ensemble 
itself. The Random Forest algorithm introduces extra randomness when growing trees; instead of searching for 
the very best feature when splitting a node, it searches for the best feature among a random subset of features. This results in a greater tree diversity, which (once again) trades a higher bias
for a lower variance, generally yielding an overall better model. The following BaggingClassifier is roughly
equivalent to the previous RandomForestClassifier
'''
bag_clf = BaggingClassifier(
                        DecisionTreeClassifier(splitter="random",	max_leaf_nodes=16),
                        n_estimators=500,	max_samples=1.0,	bootstrap=True,	n_jobs=-1
                    )


# Extra-Trees

In [11]:
'''
When you are growing a tree in a Random Forest, at each node only a random subset of the features is considered
for splitting. It is possible to make trees even more random by also using random thresholds for each feature
rather than searching for the best possible thresholds (like regular Decision Trees do). A forest of such 
extremely random trees is simply called an Extremely Randomized Trees ensemble (or Extra-Trees for short). Once
again, this trades more bias for a lower variance. It also makes Extra-Trees much faster to train than regular
Random Forests since finding the best possible threshold for each feature at every node is one of the most
time-consuming tasks of growing a tree. You can create an Extra-Trees classifier using Scikit-Learn’s 
ExtraTreesClassifier class. Its API is identical to the RandomForestClassifier class. Similarly, the 
ExtraTreesRegressor class has the same API as the RandomForestRegressor class.
'''
'''
It is hard to tell in advance whether a RandomForestClassifier will perform better or worse than an 
ExtraTreesClassifier. Generally, the only way to know is to try both and compare them using cross-validation
(and tuning the hyperparameters using grid search).
'''

'\nIt is hard to tell in advance whether a RandomForestClassifier will perform better or worse than an \nExtraTreesClassifier. Generally, the only way to know is to try both and compare them using cross-validation\n(and tuning the hyperparameters using grid search).\n'

# Feature Importance

In [12]:
'''
Lastly, if you look at a single Decision Tree, important features are likely to appear closer to the root of
the tree, while unimportant features will often appear closer to the leaves (or not at all). It is therefore
possible to get an estimate of a feature’s importance by computing the average depth at which it appears across
all trees in the forest. Scikit-Learn computes this automatically for every feature after training. You can 
access the result using the feature_importances_ variable. For example, the following code trains a 
RandomForestClassifier on the iris dataset and outputs each feature’s importance. It seems that the most 
important features are the petal length (44%) and width (42%), while sepal length and width are rather 
unimportant in comparison (11% and 2%, respectively)
'''
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name,	score)

sepal length (cm) 0.105941014329
sepal width (cm) 0.0252674302293
petal length (cm) 0.434095488569
petal width (cm) 0.434696066872


# Boosting

In [13]:
'''
Boosting (originally called hypothesis boosting) refers to any Ensemble method that can combine several weak
learners into a strong learner. The general idea of most boosting methods is to train predictors sequentially,
each trying to correct its predecessor. There are many boosting methods available, but by far the most popular
are AdaBoost (short for Adaptive Boosting) and Gradient Boosting AdaBoost
'''

'\nBoosting (originally called hypothesis boosting) refers to any Ensemble method that can combine several weak\nlearners into a strong learner. The general idea of most boosting methods is to train predictors sequentially,\neach trying to correct its predecessor. There are many boosting methods available, but by far the most popular\nare AdaBoost (short for Adaptive Boosting) and Gradient Boosting AdaBoost\n'

### AdaBoost

In [14]:
'''
One way for a new predictor to correct its predecessor is to pay a bit more attention to the training instances
that the predecessor underfitted. This results in new predictors focusing more and more on the hard cases. 
This is the technique used by AdaBoost.
'''
'''
Scikit-Learn actually uses a multiclass version of AdaBoost called SAMME (which stands for Stagewise Additive
Modeling using a Multiclass Exponential loss function). When there are just two classes, SAMME is equivalent
to AdaBoost. Moreover, if the predictors can estimate class probabilities (i.e., if they have a predict_proba() 	method),	Scikit-Learn	can	use	a	variant	of	SAMME	called
SAMME.R (the R stands for “Real”), which relies on class probabilities rather than predictions and generally
performs better.
'''
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(
                        DecisionTreeClassifier(max_depth=1), n_estimators=200,
                        algorithm="SAMME.R", learning_rate=0.5
                    )
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

### Gradient Boosting

In [18]:
'''
Another very popular Boosting algorithm is Gradient Boosting. Just like AdaBoost, Gradient Boosting works by
sequentially adding predictors to an ensemble, each one correcting its predecessor. However, instead of 
tweaking the instance weights at every iteration like AdaBoost does, this method tries to fit the new 
predictor to the residual errors made by the previous predictor. Let’s go through a simple regression example
using Decision Trees as the base predictors (of course Gradient Boosting also works great with regression 
tasks). This is called Gradient Tree Boosting, or Gradient Boosted Regression Trees (GBRT). First, let’s fit 
a DecisionTreeRegressor to the training set (for example, a noisy quadratic training set)
'''
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

# Now train a second DecisionTreeRegressor on the residual errors made by the first predictor
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

# Then we train a third regressor on the residual errors made by the second predictor
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)
'''
Now we have an ensemble containing three trees. It can make predictions on a new instance simply by adding 
up the predictions of all the trees
'''
X_new = np.array([[0.8]])
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

'''
A simpler way to train GBRT ensembles is to use Scikit-Learn’s GradientBoostingRegressor class. Much like the
RandomForestRegressor class, it has hyperparameters to control the growth of Decision Trees (e.g., max_depth ,
min_samples_leaf , and so on), as well as hyperparameters to control the ensemble training, such as the number
of trees ( n_estimators ). The following code creates the same ensemble as the previous one
'''
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X, y)

'''
The learning_rate hyperparameter scales the contribution of each tree. If you set it to a low value, such as
0.1 , you will need more trees in the ensemble to fit the training set, but the predictions will usually
generalize better. This is a regularization technique called shrinkage.
In order to find the optimal number of trees, you can use early stopping. A simple way to implement this is to 
use the staged_predict() method: it returns an iterator over the predictions made by the ensemble at each 
stage of training (with one tree, two trees, etc.). The following code trains a GBRT ensemble with 120 trees, 
then measures the validation error at each stage of training to find the optimal number of trees, and finally 
trains another GBRT ensemble using the optimal number of trees
'''
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train, y_train)
errors = [mean_squared_error(y_val, y_pred)          
            for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)
gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train, y_train)

'''
It is also possible to implement early stopping by actually stopping training early (instead of training a
large number of trees first and then looking back to find the optimal number). You can do so by setting 
warm_start=True , which makes Scikit-Learn keep existing trees when the fit() method is called, allowing 
incremental training. The following code stops training when the validation error does not improve for five 
iterations in a row
'''
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)
min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break # early stopping
            
'''
The GradientBoostingRegressor class also supports a subsample hyperparameter, which specifies the fraction of 
training instances to be used for training each tree. For example, if subsample=0.25, then each tree is trained
on 25% of the training instances, selected randomly. As you can probably guess by now, this trades a higher 
bias for a lower variance. It also speeds up training considerably. This technique is called Stochastic 
Gradient Boosting
'''

'\nThe GradientBoostingRegressor class also supports a subsample hyperparameter, which specifies the fraction of \ntraining instances to be used for training each tree. For example, if subsample=0.25, then each tree is trained\non 25% of the training instances, selected randomly. As you can probably guess by now, this trades a higher \nbias for a lower variance. It also speeds up training considerably. This technique is called Stochastic \nGradient Boosting\n'

### Stacking

In [ ]:
'''
The last Ensemble method we will discuss in this chapter is called stacking (short for stacked generalization).
It is based on a simple idea: instead of using trivial functions (such as hard voting) to aggregate the 
predictions of all predictors in an ensemble, why don’t we train a model to perform this aggregation? Each of
the bottom three predictors predicts a different value (3.1, 2.7, and 2.9), and then the final predictor 
(called a blender, or a meta learner) takes these predictions as inputs and makes the final prediction
'''
'''
To train the blender, a common approach is to use a hold-out set. Let’s see how it works. First, the training 
set is split in two subsets. The first subset is used to train the predictors in the first layer. Next, the 
first layer predictors are used to make predictions on the second (held-out) set. This ensures that the 
predictions are “clean,” since the predictors never saw these instances during training. Now for each instance
in the hold-out set there are three predicted values. We can create a new training set using these predicted
values as input features (which makes this new training set three-dimensional), and keeping the target values.
The blender is trained on this new training set, so it learns to predict the target value given the first 
layer’s predictions. It is actually possible to train several different blenders this way (e.g., one using 
Linear Regression, another using Random Forest Regression, and so on): we get a whole layer of blenders. The
trick is to split the training set into three subsets: the first one is used to train the first layer, the 
second one is used to create the training set used to train the second layer (using predictions made by the
predictors of the first layer), and the third one is used to create the training set to train the third layer
(using predictions made by the predictors of the second layer). Once this is done, we can make a prediction for
a new instance by going through each layer sequentially. Unfortunately, Scikit-Learn does not support stacking
directly, but it is not too hard to roll out your own implementation (see the following exercises). 
Alternatively, you can use an open source implementation such as brew
(available at https://github.com/viisar/brew).
'''